In [1]:
from pathlib import Path
import ffmpeg
import argparse
import numpy as np
import logging
import pandas as pd
import math
import numpy as np

In [2]:
class Sampler:
    
    def __init__(self, input_path, ann_file_path):
        """
        The Sampler class contains sampling methods for the video input data.
        :param input_path: file path for the input videos
        :param ann_file_path: file path for annotation file belonging to the video input data
        """
        self.input = input_path
        self.ann_file = ann_file_path

    
    def stratifiedSampling(self, sample_output_path, 
                           trim_len=10, n_fall_samples=3, samples_per_min=1):
        """
        Stratified sampling only samples from videos which contain all three 
        actions (ADL, falling, lying). From these videos, a base rate of fall 
        samples will be sampled uniformly on the fall interval (time where fall happens). 
        ADL and lying activities are sampled based on their ratio of duration based 
        on sample rate.
        :param sample_output_path: filepath for sample outputs
        :param label_output_path: filepath for label outputs
        :param trim_len: sample video length in seconds (defaults to 10) 
        :param fall_samples: amount of fall samples to collect from videos where 
        falls occur (defaults to 3) 
        :param samples_per_min: amount of samples per minute from ADL and lying activities 
        (defaults to 1)
        """
        
        # Read annotation file to dataframe
        df = pd.read_csv(self.ann_file)
        
        # Filter for only fall videos
        df = df[df['category'] == "Fall"] 
        
        # Reset index after filtering
        df = df.reset_index()
        
        sample_list = []

        for i in df.index:
            
            # Get timestamps
            fall_start = float(df.loc[i, 'fall_start'])
            fall_end = float(df.loc[i, 'fall_end'])
            lying_start = float(df.loc[i, 'lying_start'])
            lying_end = float(df.loc[i, 'lying_end'])
            video_end = float(df.loc[i, 'length'])
            
            # Calculate action durations
            ADL1_time = fall_start
            ADL2_time = video_end - lying_end
            ADL_time = ADL1_time + ADL2_time
            fall_time = fall_end - fall_start
            lying_time = lying_end - lying_start
            
            # Calculate number of samples for ADL and lying activities
            n_samples = round((ADL_time + lying_time) / 60) * samples_per_min
            n_ADL_samples = round(n_samples * (ADL_time / (ADL_time + lying_time)))
            n_lying_samples = n_samples - n_ADL_samples
            n_ADL1_samples = round((ADL1_time/ADL_time)*n_ADL_samples)
            n_ADL2_samples = n_ADL_samples - n_ADL1_samples
            
            # Sample uniformly on the intervals
            ADL1_samples = np.random.uniform(0, ADL1_time, n_ADL1_samples).round(decimals=3)
            ADL2_samples = np.random.uniform(lying_end, video_end, n_ADL2_samples).round(decimals=3)
            fall_samples = np.random.uniform(fall_start, fall_end, n_fall_samples).round(decimals=3)
            lying_samples = np.random.uniform(lying_start, lying_end, n_lying_samples).round(decimals=3)
            
            # Create sample list [video path, [sample timestamps]]
            sample_list.append([
                df.loc[i, "video_path"], 
                np.concatenate((ADL1_samples, 
                               ADL2_samples, 
                               fall_samples, 
                               lying_samples)).tolist()
            ])
            
        # Generate samples
        self.outputSamples(sample_list, sample_output_path)
    
    
    def outputSamples(self, sample_list, output_path, trim_len=10):
        """
        Utility function for trimming input videos and outputting them
        to the given output path (generating samples).
        :param sample_list: a list containing the video name and a list 
        of sample start timestamps, e.g. 
        [data/Fall_Simulation_Data/videos/Fall30_Cam3.avi, 
         [7.164, 15.836, 104.367, 26.325]
         ]
        :param output_path: filepath for sample outputs
        :param trim_len: sample video length in seconds (defaults to 10) 
        """
        logging.basicConfig(level=logging.INFO)
        logger = logging.getLogger(__name__)
        
        #TODO: remove
        count = 0  
        
        for sample in sample_list:
            
            # TODO: remove
            if count > 0:
                break
            count += 1
            
             # Create path
            path = Path("../" + sample[0])
            output_path = Path(output_path)
            output_path.mkdir(parents=True, exist_ok=True)
            
            # Store sample timestamps
            timestamps = sample[1]
            
            # Get video data
            video_probe = ffmpeg.probe(path)
            video_duration = video_probe.get("format", {}).get("duration", None)
            logger.debug(f"Video duration: {video_duration}")
            input_stream = ffmpeg.input(path)
            
            # Output samples
            for t in timestamps:
                
                # Trim video
                video = input_stream.trim(start=t, end=t+trim_len).setpts(
                    "PTS-STARTPTS"
                )
                
                # Create output path
                output_file_path = output_path.joinpath(
                    Path(sample[0]).stem + f"_{t}_{t+trim_len}" + ".mp4"
                )  # e.g. output_path/ADL1_Cam2_20_30.avi

                # Output
                output = ffmpeg.output(video, output_file_path.as_posix(), f="mp4")
                output.run()
                
        logger.info("Videos trimming completed successfully.")
        
        

In [3]:
sampler = Sampler("nothingfornow", "../data/Fall_Simulation_Data/annotations.csv")

sample_out = "../data/Fall_Simulation_Data/sample_outputs/"

sampler.stratifiedSampling(sample_out, samples_per_min=2)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 12.0.0
  configuration: --prefix=/Users/ktietz/demo/mc3/conda-bld/ffmpeg_1628925491858/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=arm64-apple-darwin20.0.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57

frame=  300 fps=0.0 q=-1.0 Lsize=     417kB time=00:00:09.90 bitrate= 344.7kbits/s speed=20.7x    
video:412kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.053826%
[libx264 @ 0x154015000] frame I:2     Avg QP:16.10  size: 30980
[libx264 @ 0x154015000] frame P:76    Avg QP:20.20  size:  4169
[libx264 @ 0x154015000] frame B:222   Avg QP:18.57  size:   192
[libx264 @ 0x154015000] consecutive B-frames:  1.0%  0.7%  1.0% 97.3%
[libx264 @ 0x154015000] mb I  I16..4: 35.6% 54.3% 10.1%
[libx264 @ 0x154015000] mb P  I16..4:  0.5%  0.6%  0.0%  P16..4: 28.5%  4.9%  5.4%  0.0%  0.0%    skip:60.1%
[libx264 @ 0x154015000] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 13.0%  0.0%  0.0%  direct: 0.0%  skip:86.9%  L0:51.0% L1:48.4% BI: 0.6%
[libx264 @ 0x154015000] 8x8 transform intra:54.6% inter:92.2%
[libx264 @ 0x154015000] coded y,uvDC,uvAC intra: 49.7% 60.1% 37.1% inter: 2.9% 4.5% 1.3%
[libx264 @ 0x154015000] i16 v,h,dc,p: 46% 16%  3% 36%
[libx264 @ 0x154015000] i8 v,h,dc

frame=  300 fps=0.0 q=-1.0 Lsize=     431kB time=00:00:09.90 bitrate= 356.8kbits/s speed=20.9x    
video:427kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.017656%
[libx264 @ 0x14e015000] frame I:2     Avg QP:16.16  size: 30829
[libx264 @ 0x14e015000] frame P:76    Avg QP:20.28  size:  4389
[libx264 @ 0x14e015000] frame B:222   Avg QP:18.19  size:   186
[libx264 @ 0x14e015000] consecutive B-frames:  1.0%  0.7%  1.0% 97.3%
[libx264 @ 0x14e015000] mb I  I16..4: 35.6% 54.6%  9.7%
[libx264 @ 0x14e015000] mb P  I16..4:  0.5%  0.9%  0.0%  P16..4: 29.6%  5.2%  5.7%  0.0%  0.0%    skip:58.1%
[libx264 @ 0x14e015000] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 13.0%  0.0%  0.0%  direct: 0.0%  skip:86.9%  L0:41.2% L1:58.3% BI: 0.5%
[libx264 @ 0x14e015000] 8x8 transform intra:58.3% inter:92.5%
[libx264 @ 0x14e015000] coded y,uvDC,uvAC intra: 53.6% 61.2% 38.0% inter: 3.0% 4.6% 1.3%
[libx264 @ 0x14e015000] i16 v,h,dc,p: 49% 14%  4% 34%
[libx264 @ 0x14e015000] i8 v,h,dc

frame=  300 fps=0.0 q=-1.0 Lsize=     387kB time=00:00:09.90 bitrate= 319.9kbits/s speed=17.4x    
video:382kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.136282%
[libx264 @ 0x12a015000] frame I:2     Avg QP:15.90  size: 31882
[libx264 @ 0x12a015000] frame P:76    Avg QP:19.94  size:  3863
[libx264 @ 0x12a015000] frame B:222   Avg QP:17.47  size:   151
[libx264 @ 0x12a015000] consecutive B-frames:  1.0%  0.7%  1.0% 97.3%
[libx264 @ 0x12a015000] mb I  I16..4: 35.3% 55.2%  9.5%
[libx264 @ 0x12a015000] mb P  I16..4:  0.4%  0.6%  0.0%  P16..4: 27.4%  4.6%  4.8%  0.0%  0.0%    skip:62.2%
[libx264 @ 0x12a015000] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 10.7%  0.0%  0.0%  direct: 0.0%  skip:89.3%  L0:43.1% L1:56.4% BI: 0.5%
[libx264 @ 0x12a015000] 8x8 transform intra:56.8% inter:93.5%
[libx264 @ 0x12a015000] coded y,uvDC,uvAC intra: 52.0% 60.6% 38.3% inter: 2.7% 4.0% 1.2%
[libx264 @ 0x12a015000] i16 v,h,dc,p: 45% 13%  3% 39%
[libx264 @ 0x12a015000] i8 v,h,dc

In [4]:
class Labeler:
    """
    A dedicated class for labeling our generated samples
    """

    def __init__(self, ann_file_path):
        """
        :param ann_file_path: file path for annotation file belonging to the original video input data
        """
        self.ann_file = ann_file_path
    
    
        
    def outputLabels(self, output_path):
            """
            Utility function for outputting training labels as CSV file
            to given output path.
            :param samples_path: filepath for sample putputs
            :param df: the processed dataframe from the sampling strategy
            that calls this function. 
            :param sample_list: a list containing the video name and a list 
            of sample start timestamps, e.g. 
            [data/Fall_Simulation_Data/videos/Fall30_Cam3.avi, 
             [7.164, 15.836, 104.367, 26.325]
             ]
            :param output_path: filepath for label outputs
            """

            for i in df.index:

                # Get timestamps
                fall_start = float(df.loc[i, 'fall_start'])
                fall_end = float(df.loc[i, 'fall_end'])
                lying_start = float(df.loc[i, 'lying_start'])
                lying_end = float(df.loc[i, 'lying_end'])
                video_end = float(df.loc[i, 'length'])

### 

# Saved print-outs
print('ADL1 time: ', ADL1_time)
print('ADL2 time: ', ADL2_time)
print('Total ADL time: ', ADL_time)
print('Fall time: ', fall_time)
print('Lying time: ', lying_time)

print("n samples: ", n_samples)
print("n ADL samples: ", n_ADL_samples)
print("n lying samples: ", n_lying_samples)
print("n ADL1 samples: ", n_ADL1_samples)
print("n ADL2 samples: ", n_ADL2_samples)

print("ADL1_samples: ", ADL1_samples)
print("ADL2_samples: ", ADL2_samples)
print("fall_samples: ", fall_samples)
print("lying_samples: ", lying_samples)

print(f'Video path: {df.loc[i, "video_path"]}')
print("ADL1_samples: ", ADL1_samples)
print("ADL2_samples: ", ADL2_samples)
print("fall_samples: ", fall_samples)
print("lying_samples: ", lying_samples)
print()

In [5]:
np.round(np.random.uniform(0, 2, 5), 3)

array([0.613, 0.443, 1.83 , 0.846, 0.404])